In [1]:
# ! pip install getuseragent
# ! pip install selenium

In [2]:
import os 
import pandas as pd
import numpy as np
import time
import re
import sys
from datetime import date

sys.path.append('../../')

# BeautifulSoup e Request

from bs4 import BeautifulSoup
import requests
from getuseragent import UserAgent

# selenium
from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

C:\Users\Casulippo\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
url = 'https://www.glassdoor.it/Lavoro/livorno-italia-lavori-SRCH_IL.0,14_IC2813724.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=&typedLocation=Livorno%2520(Italia)&context=Jobs&dropdown=0'

In [4]:
print(url)

https://www.glassdoor.it/Lavoro/livorno-italia-lavori-SRCH_IL.0,14_IC2813724.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=&typedLocation=Livorno%2520(Italia)&context=Jobs&dropdown=0


In [5]:
file = 'scraping_all_livorno'

In [6]:
today = date.today()

data = today.strftime("%Y%m%d")
data_file = today.strftime("%Y-%m-%d")

data, data_file

('20230217', '2023-02-17')

La data ingestion è un processo che consiste nell'ottenere e importare dati, per utilizzo immediato o l'archiviazione in un database o altri sistemi di gestione dei dati.

L'obiettivo è quello di collezionare le informazioni delle offerte di lavoro nel modo più dettagliato possibile sito www.glassdoor.com .

### Definizione delle funzioni

In [7]:
useragent = UserAgent()

theuseragent = useragent.Random()
headers = {'User-Agent': theuseragent}
header = {
    "user-agent": theuseragent ,
    'referer':'https://www.google.com/'
}

In [8]:
def allow_cookies(driver):
    print('Decline Cookies')
    button_setting_cookies = driver.find_element(By.CSS_SELECTOR, "button.cookie-setting-link")
    button_setting_cookies.click()
    time.sleep(np.random.choice([x/10 for x in range(7,22)]))    
    button_confirm_cookies = driver.find_element(By.CSS_SELECTOR, "button.save-preference-btn-handler")
    #Conferma le mie scelte
    button_confirm_cookies.click()

def next_page(driver):
    print('Changing page')
    next_button = driver.find_element(By.CSS_SELECTOR, "button.nextButton")
    next_button.click()
    url = driver.current_url

def parse_url(url):
    company_name = []
    job_title=[]
    location=[]
    company_rating=[]
    job_age=[]
    job_link=[]
    
    r = requests.get(url, headers=header)
    soup = BeautifulSoup(r.text, 'html.parser')
    job_search_div = soup.select('div#JobSearch')[0]
    a = job_search_div.select('div#PageBodyContents')
    b = a[0].select('div#JobResults')
    c = b[0].select('article#MainCol')
    len_li = len(c[0].find_all('li', class_='react-job-listing'))
    lis = c[0].find_all('li', class_='react-job-listing')
    
    len_li = len(lis)
    df = pd.DataFrame()


    for e in lis[0:len_li]:
        try:
            company_name.append(e.find('div').find('a')['title'])
        except:
            company_name.append(None)
        try:
            job_title.append(e['data-normalize-job-title'])
        except:
            job_title.append(None)    
        try:
            location.append(e['data-job-loc'])
        except:
            location.append(None)
        try:
            company_rating.append(e.find('span', class_='css-2lqh28')) ### da controllare
        except:
            company_rating.append(None)
        try:
            job_age.append(e.find("div", {"data-test": "job-age"}).text)
        except:
            job_link.append(None)
        try:
            link = "https://www.glassdoor.it" + e.find("a", {"data-test": "job-link"}).get("href")
            job_link.append(link)
        except:
            job_link.append(None)
            
    df['company'] = company_name
    df['job_title'] = job_title
    df['location'] = location
    df['company_rating']=company_rating
    df['job_age'] = job_age
    df['job_link']=job_link
    
    return df
            
def scraping_job_page(base_url, n_page=None):
    start_time = time.time()
    time_sleep = 0
    df_append = pd.DataFrame()
    driver = webdriver.Chrome(r"C:\Users\Casulippo\Desktop\web_chromedriver\chromedriver.exe")
    # base_url = 'https://www.glassdoor.it/Lavoro/amsterdam-paesi-bassi-lavori-SRCH_IL.0,21_IC3064478.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=&typedLocation=Amsterdam%2520(Paesi%2520Bassi)&context=Jobs&dropdown=0'
    driver.get(base_url)
    a = np.random.choice([x for x in range(7,15)])
    time_sleep = time_sleep + a 
    time.sleep(a)
    url = driver.current_url
    #driver.maximize_window()
    allow_cookies(driver)
    a = np.random.choice([x for x in range(7,15)])
    time_sleep = time_sleep + a 
    time.sleep(a)
    r = requests.get(url, headers=header)
    
    text = BeautifulSoup(r.text, 'html.parser').find('div', attrs={'class': 'paginationFooter'}).text
    
    int_list=[]
    for e in re.findall(r'-?\d+\.?\d*', text):
        int_list.append(int(e))
        
    if n_page == None:
        n_page = max(int_list)
    #n_page = 5
    print('Numero di pagine sui cui fare scraping: ' + str(n_page))
    a = np.random.choice([x for x in range(7,15)])
    time_sleep = time_sleep + a 
    time.sleep(a)
    
    for page in range(n_page):
        print(str(page +1) + '/' + str(n_page))
        a = np.random.choice([x for x in range(7,15)])
        time_sleep = time_sleep + a 
        time.sleep(a)
        next_page(driver)
        url = driver.current_url
        if page == 0:
            time.sleep(5)
            close_button = driver.find_element(By.CSS_SELECTOR, "svg.SVGInline-svg.modal_closeIcon-svg")
            close_button.click()
        df = parse_url(url)
        df_append = df_append.append(df).reset_index(drop=True)
    
    
    driver.quit()
    end_time = time.time()
    print('Done!')
    print('Total pages scraped:', n_page, '\n')
    print("Runtime:", round(end_time - start_time), "seconds" + '\nTime sleep:', time_sleep, 'seconds')
        
    return df_append


def scraping_company_page(links, end_n_page=None, start_n_page=None):
    start_time = time.time()
    df_append_2 =pd.DataFrame()
    time_sleep = 0
    n_link = 0
    ## lista dei link
    
    #base_url = 'https://www.glassdoor.it/job-listing/field-service-engineer-technician-lexas-JV_IC2802269_KO0,33_KE34,39.htm?jl=1008135983566&pos=101&ao=1110586&s=58&guid=000001861873f6d389d965e51ce717e2&src=GD_JOB_AD&t=SR&vt=w&uido=FEB79CBEB143B609D33C645CD13E2F4B&ea=1&cs=1_524e761d&cb=1675447498725&jobListingId=1008135983566&cpc=65CC663E25211861&jrtk=3-0-1goc77to1g2ev801-1goc77tp0i6id800-7cbed71e05432cdf--6NYlbfkN0CVO0F7mWis5ReNIXvK0Cy97GKSpj_H8mHyNoiV7tLwhxrGQFeFbXfrFFwDAnfvPXeiJe5SavTtAEQpKcpYVReYHZsV-4ZX7UeAkoBb0f_WCVWviQdPDhB0WcxVHddsJTu6CPWu9hRPncXvGLdy3ZffF5b3aOd7vp19QcNQdw0qQd1bkijbQHvL2CZX_Cxp4BGS1Sk8JgAjiz75HrAHRuR5hA9kjnxafzWfGAAOJBSKybBbJtFcKCvWC2Py0-IgF36KHcIY5QbFzm8TqI0WJJ75VyN8D93fcG7Ikeu9ECT-vBbPKtsVv7AOpI6elTa0KTJqDzeYeI39a8bHa5BUUOjrQ1rsJFwMGLJrLrbqXOIObHs1pSu_fpr-FcNbNcmvwdLQufgm_hOEka1AR5pS2jw3Kd3MEOLDniaBdQ1tk-tgoNuL5lhZNjQUe648ZgRuUWrLaIX3oHWCh15jyNqf9dfjjRB70aXgFCI2nC397dI72pLnPhLIqqOKxPTEV4El04FFBbjsdOPDp3q0LQurvyiZmQB21o9E3WEoCcol53Fx2g0SIc5ixunqT-a4RIeTz8CzvY7DdPxvqKSbekvmaTBa_UWB3gbtD9Y%253D&ctt=1675447519123'
       
    
    if start_n_page == None:
        start_n_page=0
    if end_n_page == None:
        end_n_page=len(links)
        
    print('link to scrape: ', end_n_page - start_n_page) 
    
    for base_url in links[start_n_page:end_n_page]: 
        n_link += 1
        driver = webdriver.Chrome(r"C:\Users\Casulippo\Desktop\web_chromedriver\chromedriver.exe")
        driver.get(base_url)
        a = np.random.choice([x for x in range(7,15)])
        time_sleep = time_sleep + a 
        time.sleep(a)
        ## cookies
        allow_cookies(driver)
        a = np.random.choice([x for x in range(7,15)])
        time_sleep = time_sleep + a 
        time.sleep(a)
        ## Mapping dei bottoni
        general_button = driver.find_elements(By.CSS_SELECTOR, ".css-dkrzi8.e1eh6fgm0")
        button_mapping = {valore.text: indice for indice, valore in enumerate(general_button)}
    
        
        d_valutazione={}
        d_azienda={}
    
        print('check the buttons-->', n_link)
        try:
            general_button = driver.find_elements(By.CSS_SELECTOR, ".css-dkrzi8.e1eh6fgm0")
            
        except:
            print('no buttons found')
            general_button = []
        
        
        if len(general_button)>0:
            
            try:
    
                azienda = general_button[button_mapping['Azienda']] # Azienda
                azienda.click()
                a = np.random.choice([x for x in range(7,15)])
                time_sleep = time_sleep + a 
                time.sleep(a)
                pag_azienda_span = driver.find_elements(By.CSS_SELECTOR, ".css-vugejy.es5l5kg0 span.value")
                pag_azienda_value = driver.find_elements(By.CSS_SELECTOR, ".css-vugejy.es5l5kg0 label")
                
                azienda_span = []
                azienda_value = []
                
                for e in pag_azienda_span:
                    azienda_span.append(e.text)
                
                for e in pag_azienda_value:
                    azienda_value.append(e.text)
                d_azienda = dict(zip(azienda_value, azienda_span))
                print('scraped company page')
            
                
            except:
                pass
            
            try:
                valutazione = general_button[button_mapping['Valutazione']] # Valutazione
                #recensioni = general_button[2] # Recensioni
                
                ## Cambio foglio 
                valutazione.click()
                pag_valutazione = driver.find_elements(By.CSS_SELECTOR, ".css-a7hxlj.e121l59f1")
                a = np.random.choice([x for x in range(7,15)])
                time_sleep = time_sleep + a 
                time.sleep(a)
                valutazione_1 = [pag_valutazione[0].text, pag_valutazione[2].text, pag_valutazione[4].text,pag_valutazione[6].text, pag_valutazione[8].text]
                valutazione_2 = [pag_valutazione[1].text, pag_valutazione[3].text, pag_valutazione[5].text,pag_valutazione[7].text, pag_valutazione[9].text]
                print('scraped evaluation page')
    
                d_valutazione = dict(zip(valutazione_1, valutazione_2))
            
            except:
                pass  
            # to add benefit e stipendio
    
                
            
        d = {**d_valutazione, **d_azienda}
            
        driver.quit()
        
        
        df2 = pd.DataFrame.from_dict(d,  orient='index').T
        df2['job_link']=base_url
        
        df_append_2 = df_append_2.append(df2).reset_index(drop=True)

    
    end_time = time.time()
    print('Total pages scraped:', len(links), '\n')
    print("Runtime:", round(end_time - start_time), "seconds" + '\nTime sleep: ', time_sleep, 'seconds')
    return df_append_2

Una volta definite le funzioni per procedere con lo scraping, ci saranno due faso di scraping:
1. Scraping delle offerte di lavoro 
2. Scraping delle aziende che offrono un lavoro

### 1.Scraping delle offerte di lavoro

Questa prima fase si occuperà di fare scraping sulle offerte di lavoro relative ad un determinato link.
(es [offerte lavoro amsterdam]('https://www.glassdoor.it/Lavoro/amsterdam-paesi-bassi-lavori-SRCH_IL.0,21_IC3064478.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=&typedLocation=Amsterdam%2520(Paesi%2520Bassi)&context=Jobs&dropdown=0'))


Se il link presenta più pagine relative alle offerte cercate, il sistema andrà a fare lo scraping su tutte le pagine sino ad arrivare alla 30°.  
E' possibile limitare il numero di pagine sui fare scraping con l'argomento n_page, di default vengono prese tutte le pagine disponibile (max 30).  
Dopo aver avviato la navigazione web il sistema rifiuterà i cookies e inizierà lo scraping per tutte le pagine presenti nel link.  
Una volta completato lo scraping verrà creato un dataframe con le informazioni raccolte. Il dataframe presenterà queste colonne:
- company: nome dell'azienda
- job_title: Posizione lavorativa offerta
- location: Sede dell'offerta di lavoro
- company_rating: rating dell'azienda (secondo gli utenti di Glassdoor)
- job_age: giorni passati dalla pubblicazione dell'offerta
- job_link: link dell'azienda che ha pubblicato il lavoro


Si farà adesso lo scraping delle prime due pagine del link [offerte lavoro amsterdam]('https://www.glassdoor.it/Lavoro/amsterdam-paesi-bassi-lavori-SRCH_IL.0,21_IC3064478.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=&typedLocation=Amsterdam%2520(Paesi%2520Bassi)&context=Jobs&dropdown=0')

In [9]:
df_job_scraping = scraping_job_page(url)

<ipython-input-8-317a0319a6df>:77: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\Casulippo\Desktop\web_chromedriver\chromedriver.exe")


Decline Cookies
Numero di pagine sui cui fare scraping: 30
1/30
Changing page
2/30
Changing page
3/30
Changing page
4/30
Changing page
5/30
Changing page
6/30
Changing page
7/30
Changing page
8/30
Changing page
9/30
Changing page
10/30
Changing page
11/30
Changing page
12/30
Changing page
13/30
Changing page
14/30
Changing page
15/30
Changing page
16/30
Changing page
17/30
Changing page
18/30
Changing page
19/30
Changing page
20/30
Changing page
21/30
Changing page
22/30
Changing page
23/30
Changing page
24/30
Changing page
25/30
Changing page
26/30
Changing page
27/30
Changing page
28/30
Changing page
29/30
Changing page
30/30
Changing page
Done!
Total pages scraped: 30 

Runtime: 433 seconds
Time sleep: 361 seconds


In [10]:
df_job_scraping.head()

,company,job_title,location,company_rating,job_age,job_link
0,QUIRICONI PROGETTI SRL - SOCIETA' DI INGEGNERIA,Architetto Navale,Italia,None,Oltre 30 g,https://www.glassdoor.it/partner/jobListing.ht...
1,GIGROUP,Receptionist,Italia,"[3.8, []]",17 g,https://www.glassdoor.it/partner/jobListing.ht...
2,Takeda Pharmaceutical,Lead Maintenance Technician,Italia,"[4, []]",3 g,https://www.glassdoor.it/partner/jobListing.ht...
3,CISALPINA TOURS,Senior Travel Consultant,Italia,None,Oltre 30 g,https://www.glassdoor.it/partner/jobListing.ht...
4,Seitel,Operatore,Italia,None,25 g,https://www.glassdoor.it/partner/jobListing.ht...


In [11]:
# con questa cella è possibile salvare il file nella cartella data
df_job_scraping['run_date'] = data_file


file_name = file + '_job_' + data + '.csv'
df_job_scraping.to_csv('../data/' + file_name, index=False)

### 2. Scraping delle aziende che offrono un lavoro

Una volta completato la prima fase di scraping, e' possibile passare alla seconda.  

Questa seconda fase prevede la ricerca di informazioni delle aziende che offrono un lavoro, queste verrano cercate nella sezione dedicata creata da Glassdoor.  

Si compierà uno scraping delle informazioni generali e delle valutazioni date dagli utenti di Glassdoor.  


Per fare questo utilizziamo la colonna "job_link" (*link dell'azienda che ha pubblicato il lavoro*) che trasformiamo in una lista da passare alla funzione "scraping_company_page".


Una volta completato lo scraping verrà creato un dataframe con le informazioni raccolte. Il dataframe presenterà queste colonne:
- Opportunità di carriera (rating)
- Stipendio e benefit (rating)
- Cultura e valori (rating)
- Dirigenti senior (rating)
- Equilibrio lavoro/vita privata (rating)
- Sede centrale	
- Dimensioni	
- Fondata nel	
- Tipo	
- Settore Segmento	
- Entrate
- job_link (chiave primaria)

Sarà possibile unire i due dataframe con la chiave job_link.

Anche questa funzione prende l'argomento n_page per limitare il numero di pagine sui cui fare scraping.

In [12]:
#file_name = file + '_job_' + data + '.csv'
#df_job_scraping = pd.read_csv('../data/' + file_name)

In [13]:
path_overview = '../data/company_overview.csv'
df_company_overview = pd.read_csv(path_overview)
old_links = list(df_company_overview['job_link'].unique())
links = list(df_job_scraping['job_link'].unique())
new_links = list(set(links) - set(old_links))

In [14]:
df_scraping_company=pd.DataFrame()
df_fitt=pd.DataFrame()
n = 5
start_to=0
start_time = time.time()
for i in range(start_to, len(links), n):
    try:
        df_fitt = scraping_company_page(links, end_n_page = i+n, start_n_page=i)
        df_scraping_company = df_scraping_company.append(df_fitt).reset_index(drop=True)
        print(len(df_scraping_company))
    except:
        print('Qualche errore sconosciuto, riparto..\n')
        print(len(df_scraping_company))
        pass
end_time = time.time()
print("Runtime:", round(end_time - start_time), "seconds")

link to scrape:  5


<ipython-input-8-317a0319a6df>:148: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\Casulippo\Desktop\web_chromedriver\chromedriver.exe")


Decline Cookies
check the buttons--> 1
Decline Cookies
check the buttons--> 2
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 3
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 4
Decline Cookies
check the buttons--> 5
Total pages scraped: 900 

Runtime: 182 seconds
Time sleep:  133 seconds
2
link to scrape:  5
Decline Cookies
check the buttons--> 1
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 2
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 3
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 4
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 5
scraped company page
scraped evaluation page
Total pages scraped: 900 

Runtime: 286 seconds
Time sleep:  230 seconds
7
link to scrape:  5
Decline Cookies
check the buttons--> 1
Decline Cookies
check the buttons--> 2
Decline Cookies
check the but

55
link to scrape:  5
Decline Cookies
check the buttons--> 1
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 2
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 3
Decline Cookies
check the buttons--> 4
Decline Cookies
check the buttons--> 5
scraped company page
scraped evaluation page
Total pages scraped: 900 

Runtime: 220 seconds
Time sleep:  166 seconds
58
link to scrape:  5
Decline Cookies
check the buttons--> 1
Decline Cookies
check the buttons--> 2
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 3
Decline Cookies
check the buttons--> 4
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 5
scraped company page
scraped evaluation page
Total pages scraped: 900 

Runtime: 219 seconds
Time sleep:  166 seconds
61
link to scrape:  5
Decline Cookies
check the buttons--> 1
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 2
Decli

scraped evaluation page
Decline Cookies
check the buttons--> 3
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 4
Decline Cookies
check the buttons--> 5
Total pages scraped: 900 

Runtime: 203 seconds
Time sleep:  151 seconds
110
link to scrape:  5
Decline Cookies
check the buttons--> 1
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 2
Decline Cookies
check the buttons--> 3
Decline Cookies
check the buttons--> 4
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 5
Total pages scraped: 900 

Runtime: 209 seconds
Time sleep:  159 seconds
112
link to scrape:  5
Decline Cookies
check the buttons--> 1
Decline Cookies
check the buttons--> 2
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 3
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 4
Decline Cookies
check the buttons--> 5
Total pages scraped: 900 

Runtime: 208 seconds
Time

scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 5
scraped company page
scraped evaluation page
Total pages scraped: 900 

Runtime: 202 seconds
Time sleep:  149 seconds
168
link to scrape:  5
Decline Cookies
check the buttons--> 1
Decline Cookies
check the buttons--> 2
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 3
Decline Cookies
check the buttons--> 4
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 5
Total pages scraped: 900 

Runtime: 200 seconds
Time sleep:  153 seconds
170
link to scrape:  5
Decline Cookies
check the buttons--> 1
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 2
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 3
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 4
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 5
scraped company pag

check the buttons--> 5
Total pages scraped: 900 

Runtime: 200 seconds
Time sleep:  147 seconds
225
link to scrape:  5
Decline Cookies
check the buttons--> 1
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 2
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 3
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 4
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 5
scraped company page
scraped evaluation page
Total pages scraped: 900 

Runtime: 282 seconds
Time sleep:  207 seconds
230
link to scrape:  5
Decline Cookies
check the buttons--> 1
Decline Cookies
check the buttons--> 2
Decline Cookies
check the buttons--> 3
Decline Cookies
check the buttons--> 4
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 5
scraped company page
scraped evaluation page
Total pages scraped: 900 

Runtime: 198 seconds
Time sleep:  139 seconds
232
lin

Total pages scraped: 900 

Runtime: 259 seconds
Time sleep:  206 seconds
284
link to scrape:  5
Decline Cookies
check the buttons--> 1
Decline Cookies
check the buttons--> 2
Decline Cookies
check the buttons--> 3
Decline Cookies
check the buttons--> 4
scraped company page
scraped evaluation page
Decline Cookies
check the buttons--> 5
scraped company page
scraped evaluation page
Total pages scraped: 900 

Runtime: 209 seconds
Time sleep:  140 seconds
286
link to scrape:  5
Decline Cookies
check the buttons--> 1
scraped company page
scraped evaluation page
Decline Cookies


NameError: name 'driver' is not defined

In [15]:
# con questa cella è possibile salvare il file nella cartella data
df_scraping_company = df_scraping_company.rename(columns={'Sede centrale':'sede_centrale',
                                'Dimensioni':'dimensioni',
                                'Fondata nel' : 'fondata_nel',
                                'Tipo':'tipo',
                                'Entrate':'entrate',
                                # job_link
                                'Opportunità di carriera': 'oppurtunita_carriera',
                                'Stipendio e benefit':'stipendio_e_benefit',
                                'Cultura e valori':'cultura_e_valori',
                                'Dirigenti senior':'dirigenti_senior',
                                'Equilibrio lavoro/vita privata':'equilibrio_lavoro_vita_privata',
                                'Settore':'settore',
                                'Segmento':'segmento'})
#file_name = file + '_company_' + data + '.csv'
#df_scraping_company.to_csv('../data/' + file_name, index=False)

In [16]:
df_company_overview_new = df_company_overview.append(df_scraping_company).drop_duplicates().reset_index(drop=True)

In [17]:
df_company_overview_new

,job_link,oppurtunita_carriera,stipendio_e_benefit,cultura_e_valori,dirigenti_senior,equilibrio_lavoro_vita_privata,sede_centrale,dimensioni,tipo,entrate,fondata_nel,settore,segmento
0,https://www.glassdoor.it/partner/jobListing.ht...,2,3,4,1,3,"Napoli, Italia",Sconosciuto,Azienda quotata in borsa,Sconosciuto/Non applicabile,NaN,NaN,NaN
1,https://www.glassdoor.it/partner/jobListing.ht...,3.6,3.4,3.3,3,2.7,"Roma, Italia",Oltre 10.000 dipendenti,Azienda - Settore privato,$ 1 - $ 5 miliardi (USD),1988,Servizi di sicurezza e protezione,Management e consulenza
2,https://www.glassdoor.it/partner/jobListing.ht...,3.3,3.1,4,3.4,3.8,"Milano, Italia",Sconosciuto,Azienda quotata in borsa,Sconosciuto/Non applicabile,NaN,NaN,NaN
3,https://www.glassdoor.it/partner/jobListing.ht...,NaN,NaN,NaN,NaN,NaN,"Arezzo, Italia",Sconosciuto,Azienda quotata in borsa,Sconosciuto/Non applicabile,NaN,NaN,NaN
4,https://www.glassdoor.it/partner/jobListing.ht...,2.9,2.8,3.2,2.8,2.7,"Venezia, Italia",Oltre 10.000 dipendenti,Azienda - Settore privato,$ 100 - $ 500 milioni (USD),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1533,https://www.glassdoor.it/partner/jobListing.ht...,3.5,3.2,3.6,3.3,3.5,"Zürich, Svizzera",Da 201 a 500 dipendenti,Azienda quotata in borsa,Più di $ 10 miliardi (USD),1996,Selezione del personale e subappalti,Risorse umane e selezione del personale
1534,https://www.glassdoor.it/partner/jobListing.ht...,3.2,3.3,3.7,3.4,3.5,"Milano, Italia",Oltre 10.000 dipendenti,Azienda - Settore privato,$ 1 - $ 5 miliardi (USD),1998,Consulenza HR,Risorse umane e selezione del personale
1535,https://www.glassdoor.it/partner/jobListing.ht...,3.4,3.4,3.8,3.3,3.9,"Geneva, Svizzera",Oltre 10.000 dipendenti,Azienda quotata in borsa,Più di $ 10 miliardi (USD),1987,Produzione elettronica,Manifattura
1536,https://www.glassdoor.it/partner/jobListing.ht...,3.5,3.2,3.6,3.3,3.5,"Zürich, Svizzera",Da 201 a 500 dipendenti,Azienda quotata in borsa,Più di $ 10 miliardi (USD),1996,Selezione del personale e subappalti,Risorse umane e selezione del personale


In [18]:
df_company_overview_new.to_csv(path_overview, index=False)

E' possibile adesso mergiare i due file ed ottenere una il dataset finale da poter valutare dal punto di vista qualitativo

In [19]:
df_final = df_job_scraping.merge(df_company_overview_new, how='left', on='job_link')
df_final['company_rating'] = df_final['company_rating'].str.extract(r'>\s*([0-9.]+)\s*<')
# con questa cella è possibile salvare il file nella cartella data
float_columns = [
        'company_rating','oppurtunita_carriera', 'stipendio_e_benefit',
       'cultura_e_valori', 'dirigenti_senior',
       'equilibrio_lavoro_vita_privata']
string_columns = [
    'company', 'job_title', 'location', 'job_age',
       'job_link', 'sede_centrale', 'dimensioni', 'tipo', 'settore', 'segmento', 'entrate']
integer_columns = ['fondata_nel']

for e in float_columns:
    df_final[e] = df_final[e].astype(float)

for e in string_columns:
    df_final[e] = df_final[e].astype(str)
    
for e in integer_columns:
    df_final[e] = df_final[e].fillna(0).astype(int).replace(0,None)

In [20]:
df_final

,company,job_title,location,company_rating,job_age,job_link,run_date,oppurtunita_carriera,stipendio_e_benefit,cultura_e_valori,dirigenti_senior,equilibrio_lavoro_vita_privata,sede_centrale,dimensioni,tipo,entrate,fondata_nel,settore,segmento
0,QUIRICONI PROGETTI SRL - SOCIETA' DI INGEGNERIA,Architetto Navale,Italia,NaN,Oltre 30 g,https://www.glassdoor.it/partner/jobListing.ht...,2023-02-17,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,0,nan,nan
1,GIGROUP,Receptionist,Italia,NaN,17 g,https://www.glassdoor.it/partner/jobListing.ht...,2023-02-17,3.2,3.3,3.7,3.4,3.5,"Milano, Italia",Oltre 10.000 dipendenti,Azienda - Settore privato,$ 1 - $ 5 miliardi (USD),1998,Consulenza HR,Risorse umane e selezione del personale
2,Takeda Pharmaceutical,Lead Maintenance Technician,Italia,NaN,3 g,https://www.glassdoor.it/partner/jobListing.ht...,2023-02-17,3.7,4.0,4.1,3.6,3.8,"Tokyo, Giappone",Oltre 10.000 dipendenti,Azienda - Settore privato,Più di $ 10 miliardi (USD),1781,Biotecnologia e farmaceutica,Farmaceutico e biotecnologico
3,CISALPINA TOURS,Senior Travel Consultant,Italia,NaN,Oltre 30 g,https://www.glassdoor.it/partner/jobListing.ht...,2023-02-17,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,1781,nan,nan
4,Seitel,Operatore,Italia,NaN,25 g,https://www.glassdoor.it/partner/jobListing.ht...,2023-02-17,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,1781,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,adecco,Ingegnere Meccanico,Italia,NaN,22 g,https://www.glassdoor.it/partner/jobListing.ht...,2023-02-17,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,1988,nan,nan
896,REALTEK1,Industrial Designer,Italia,NaN,Oltre 30 g,https://www.glassdoor.it/partner/jobListing.ht...,2023-02-17,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,1988,nan,nan
897,Cisa Group,Ingegnere di Configurazione,Italia,NaN,Oltre 30 g,https://www.glassdoor.it/partner/jobListing.ht...,2023-02-17,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,1988,nan,nan
898,Miroglio Group,Store Manager,Italia,NaN,Oltre 30 g,https://www.glassdoor.it/partner/jobListing.ht...,2023-02-17,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,1988,nan,nan


In [21]:
file_name = file + '_' + data + '.csv'
df_final.to_csv('../data/' + file_name, index=False)

####### RICORDA di committare